# Deduplication Chunking in a Nutshell

## Introduction

Data deduplication is a technique for eliminating duplicate copies of repeating data. A related and somewhat synonymous term is single-instance (data) storage. (from: [dedup@wikipedia](https://en.wikipedia.org/wiki/Data_deduplication))

This notebook will introduce chunking methods and demonstrate how it works. It is divided into two parts: part 1 - mini deduplication demo, part 2 - chunking methods demo.

The example code is minimalistic. I wrote as little code as possible so anyone can play with it and understand some implications of chosing a chunking method and its algorithm's parameters.

## Deduplication in a Nutshell

What deduplication does in a nutshell is:

  1. Divide data chunks
  2. Calculate the chunks' hashes
  3. Store chunks uniquely
  
I will ignore the complexities of building such a system for actual big data, and write a simple key value storage class that saves so-called unique data only.

In [1]:
import hashlib

class DeduplicationStorage:
    def __init__(self, chunk_function):
        self.chunk = chunk_function
        self.hash = lambda raw, f=hashlib.md5: f(raw).digest()
        self.chunks = {}
        self.objects = {}

    def write(self, key, value):
        chunks = list(self.chunk(value))
        hashes = [self.hash(raw) for raw in chunks]
        self.chunks.update(zip(hashes, chunks))
        self.objects[key] = hashes
    
    def read(self, key):
        hashes = self.objects[key]
        return b''.join(self.chunks[h] for h in hashes)
        
    def print_stats(self):
        data_size = sum(len(self.chunks[h]) for hashes in self.objects.values() for h in hashes)
        dedup_size = sum(len(raw) for raw in self.chunks.values())
        print(f'Data Actual Size = {data_size} bytes.')
        print(f'Deduplicated Size = {dedup_size} bytes.')

## Chunking Methods

Breifly chunking methods can be divided into 3 different categories:

  1. Simple - for example fixed size.
  2. Content aware - for example: files, objects.
  3. Content sensitive - using a [rolling hash](https://en.wikipedia.org/wiki/Rolling_hash) to find uniquely defined boundaries.
 
Below there is actual code for `chunking` data, and an axample section showing how it all fits together.

### helper

This will be used by all other functions to split the raw data besed on chunk boundary offsets.

In [2]:
def split_by_offsets(raw, offsets):
    p = 0
    for offset in offsets:
        yield raw[p:offset]
        p = offset
    if p < len(raw):
        yield raw[p:]

### Chunking Method - Simple
The following function chunk the raw data into fixed size chunks of data.

In [3]:
def chunk_by_size(raw, size=32):
    offsets = range(size, len(raw), size)
    return split_by_offsets(raw, offsets)

### Chunking Method - Content Aware
The following function chunk the raw data into sentences, assuming the input raw data is a text. I used *text* and *sentences* since it is easy to reason about and inpect.

In [4]:
import re
def chunk_by_sentence(raw):
    offsets = (match.end() for match in re.finditer(b'\. ', raw))
    return split_by_offsets(raw, offsets)

### Chunking Method - Content Sensetive
The following function chunk the raw data into variable size chunks based on a hash value.
Typical products will use rolling hash, but for this example I found no reason to complicate the code which is inefficient to begin with.

The idea of hash based chunking, is to mark boudaries in areas which some random criteria applies on. In this way the boundaries of the same data will be in the same places even if the data is inserted into a different context.

In [5]:
import hashlib

WINDOW_SIZE = 4
HASH = lambda raw: hashlib.md5(raw).digest()

def chunk_by_hash(raw):
    offsets = [
        i + 1
        for i in range(WINDOW_SIZE, len(raw))
        if (HASH(raw[i - WINDOW_SIZE:i])[0] & 31) == 31
    ]
    return split_by_offsets(raw, offsets)

## Exmaple - How does it all fit together

To get the feeling of how chunking works I will use the first paragraph from "Phantom of the Opera".
I will store this paragraph in the demo-deduplication repository twice. The original version and a modified version changing the word "absurd" with "joy".

What we will see in this example is the effect the chunking method have on a deduplication repository. We can then inspect the chunking methods to see why it effect the repository so greatly.

### Defining Raw Data

In [6]:
original = (b'THE Opera ghost really existed. He was not, as was long believed, a cr'
 b'eature of the imagination of the artists, the superstition of the managers, '
 b'or a product of the absurd and impressionable brains of the young ladies of '
 b'the ballet, their mothers, the box-keepers, the cloak-room attendants or the'
 b' concierge. Yes, he existed in flesh and blood, although he assumed the comp'
 b'lete appearance of a real phantom; that is to say, of a spectral shade.')

modified = original.replace(b'absurd', b'joy')

### Testing our "Deduplication"

In [7]:
for name, func in (
    ('size', chunk_by_size),
    ('sentence', chunk_by_sentence),
    ('hash', chunk_by_hash),
):
    print(f'\nDedup test using chunk by {name}')
    dedup_store = DeduplicationStorage(func)

    dedup_store.write('org', original)
    dedup_store.write('mod', modified)

    assert original == dedup_store.read('org')

    dedup_store.print_stats()


Dedup test using chunk by size
Data Actual Size = 887 bytes.
Deduplicated Size = 727 bytes.

Dedup test using chunk by sentence
Data Actual Size = 887 bytes.
Deduplicated Size = 720 bytes.

Dedup test using chunk by hash
Data Actual Size = 887 bytes.
Deduplicated Size = 509 bytes.


I run 3 versions of this test. In each version we use a different chunking method and count the amount of actual data needed to be stored in the deduplicated so-called repository and the actual data size given by the user.

I intentionally ignored any metadata and other artifacts. It is usually the case that the smaller the chunk size is the more metadata per any mount of user data grows. An actual deduplication system architecture is complicated and looks nothing like this example. Compacting the metadata is a different story I will not cover here.

As you can see, in this case chunking yield significantly better data reduction results.
It is the usual case when the deduplication system architect have no prior knowledge on the information the raw data represent and how it represents it.

### Inspecting the chunking methods

Now I would like to take a look at each one of those methods and see how the chunks look like. I defined a short that creates a short representation of a chunk in a human readable text. The original include short hash of the chunk, the size of the chunk, and the first 10 chrecters of the chunk.

In [8]:
# a helper short hash representation of chunks
short = lambda x: f'{hex(hash(x))[-6:]} {len(x): 3d} {x.decode()[:10]}{"..." if len(x) > 10 else ""}'

### Fixed Size
now lets take a look at the simple method of chunking by size, by inpsecting the original paragraph chunks and the modified paragraph's chunks with an indication for each chunk in the modified paragraph for if it can be found in the original paragraph after it was chunked.

In [9]:
print("Original:")
print('\n'.join(short(chunk) for chunk in chunk_by_size(original)))
isdup = lambda c, cs = set(chunk_by_size(original)): 'dup' if c in cs else 'new'
print('\n'.join(f'{isdup(chunk)} {short(chunk)}' for chunk in chunk_by_size(modified)))

Original:
fd180d  32 THE Opera ...
f5986b  32 He was not...
3f82df  32 , a creatu...
faf507  32 of the art...
9d76d9  32  of the ma...
3162eb  32 f the absu...
65b2f7  32 brains of ...
622d66  32 e ballet, ...
2fab5f  32 -keepers, ...
15e51c  32 nts or the...
9a3b5a  32 isted in f...
9a5725  32 gh he assu...
41d665  32 rance of a...
1eab05  29  to say, o...
dup fd180d  32 THE Opera ...
dup f5986b  32 He was not...
dup 3f82df  32 , a creatu...
dup faf507  32 of the art...
dup 9d76d9  32  of the ma...
new 43ff5a  32 f the joy ...
new eba3e0  32 ins of the...
new 1e2c09  32 allet, the...
new 6a0867  32 epers, the...
new 9b2eba  32  or the co...
new 34b50b  32 ed in fles...
new 6e058f  32 he assumed...
new 3150bc  32 ce of a re...
new 1e6e71  26  say, of a...


As we can see all the chunks up to the point of the change can be found in both versions of the paragraph. However from the change and on, nothing is identified as duplication.

This is because the replaced word is of different size, which shifted the entire text afterwards. The fixed size method is known to be inefficient for any data that can be shifted.

In some cases the fixed size method works best. Some applications optimize for block storage, if those products are the owner of the data you want to deduplicate it would be wise to use the very same block size your application uses. In other cases, fixed size chunking is used due to other limitations or requirements from the designed deduplication system.

### Content Aware

I will demonstrate this awareness by using identifying sentences.
Lets see how it works.

In [10]:
print("Original:")
print('\n'.join(short(chunk) for chunk in chunk_by_sentence(original)))
isdup = lambda c, cs = set(chunk_by_sentence(original)): 'dup' if c in cs else 'new'
print('\n'.join(f'{isdup(chunk)} {short(chunk)}' for chunk in chunk_by_sentence(modified)))

Original:
fd180d  32 THE Opera ...
1571d0  278 He was not...
a99641  135 Yes, he ex...
dup fd180d  32 THE Opera ...
new 50c385  275 He was not...
dup a99641  135 Yes, he ex...


Here we can see that the only changed chunk was the chunk representing the sentence that was changed.
This method is useful when the user of the deduplication system saved objects that can be identified as a whole and both thier size and changes are big enough to ignore partial changes or group it in case it is too small.

### Content Sensitive

This is the most robust method, since you need not to assume nothing about the input. In this example it works best.

In [11]:
print("Original:")
print('\n'.join(short(chunk) for chunk in chunk_by_sentence(original)))
isdup = lambda c, cs = set(chunk_by_sentence(original)): 'dup' if c in cs else 'new'
print('\n'.join(f'{isdup(chunk)} {short(chunk)}' for chunk in chunk_by_sentence(modified)))

Original:
fd180d  32 THE Opera ...
1571d0  278 He was not...
a99641  135 Yes, he ex...
dup fd180d  32 THE Opera ...
new 50c385  275 He was not...
dup a99641  135 Yes, he ex...


In this example this method yields the smallest change.

One intersting thing here is that there are two new chunks in the modified paragraph, but only one in the original. The word changed, as small as it may be, added a new boundary.

considering other methods, when using fixed size it cannot happen, and when using content aware it can only happen when the semantics of the content are changed, in this example if we break a sentence in the middle adding a dot, a new boundary will appear.

To play with the size of the chunk, you can try and play with the criteria. In this code we check if 5 bits are all 1. we can actually compare the to any number with same number of bits. So the average chunk size should be 2^5, so if we want larger chunk sizes we'll add bits, or remove bits, or redefine any criteria that suits our needs.

## Summary

I hope this will help you understand chunking methods, and the motivation of using hashes to chunk data for deduplication.

Cheers,<br/>
Johnny D.